In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
# import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [7]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()
#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'input/train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

In [8]:
print(data.shape, labels.shape)
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)


(39209, 30, 30, 3) (39209,)
(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [9]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

In [10]:
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")


Epoch 1/15
981/981 [==============================] - 33s 33ms/step - loss: 2.1106 - accuracy: 0.4797 - val_loss: 0.5825 - val_accuracy: 0.8681
Epoch 2/15
981/981 [==============================] - 34s 35ms/step - loss: 0.7240 - accuracy: 0.7873 - val_loss: 0.2382 - val_accuracy: 0.9433
Epoch 3/15
981/981 [==============================] - 35s 36ms/step - loss: 0.4378 - accuracy: 0.8742 - val_loss: 0.2170 - val_accuracy: 0.9361
Epoch 4/15
981/981 [==============================] - 34s 35ms/step - loss: 0.3611 - accuracy: 0.8971 - val_loss: 0.1133 - val_accuracy: 0.9668
Epoch 5/15
981/981 [==============================] - 32s 33ms/step - loss: 0.3015 - accuracy: 0.9142 - val_loss: 0.1101 - val_accuracy: 0.9681
Epoch 6/15
981/981 [==============================] - 32s 33ms/step - loss: 0.2640 - accuracy: 0.9265 - val_loss: 0.1114 - val_accuracy: 0.9699
Epoch 7/15
981/981 [==============================] - 32s 33ms/step - loss: 0.2434 - accuracy: 0.9321 - val_loss: 0.0751 - val_accuracy:

Plotando em graficos


In [ ]:
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [17]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('input/Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
predict_x=model.predict(X_test) 
#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, predict_x))
model.save('traffic_classifier.h5')

395/395 [==============================] - 2s 5ms/step


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets